In [1]:
# create table public_for_2324.tt_tag_proportion as
#  select 'python' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#    from (
#         select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#          from public_for_2324.posts
#          where posttypeid = '1'
#            and creationdate between '2021-11-30' and '2024-12-02'
#            and tags like '%<python>%'
#             ) xx,
#         (
#             select x.id, count(x.tag) as tot_cnt
#                from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                        from public_for_2324.posts
#                      where posttypeid = '1'
#                        and creationdate between '2021-11-30' and '2024-12-02'
#                        and tags like '%<python>%') x
#              group by x.id

#             )yy
#  where xx.id = yy.id
#  ;


# insert into public_for_2324.tt_tag_proportion
#  select 'c++' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#    from (
#         select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#          from public_for_2324.posts
#          where posttypeid = '1'
#            and creationdate between '2021-11-30' and '2024-12-02'
#            and tags like '%<c++>%'
#             ) xx,
#         (
#             select x.id, count(x.tag) as tot_cnt
#                from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                        from public_for_2324.posts
#                      where posttypeid = '1'
#                        and creationdate between '2021-11-30' and '2024-12-02'
#                        and tags like '%<c++>%') x
#              group by x.id

#             )yy
#  where xx.id = yy.id
#  ;

# insert into public_for_2324.tt_tag_proportion
#   select 'java' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#     from (
#          select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#           from public_for_2324.posts
#           where posttypeid = '1'
#             and creationdate between '2021-11-30' and '2024-12-02'
#             and tags like '%<java>%'
#              ) xx,
#          (
#              select x.id, count(x.tag) as tot_cnt
#                 from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                         from public_for_2324.posts
#                       where posttypeid = '1'
#                         and creationdate between '2021-11-30' and '2024-12-02'
#                         and tags like '%<java>%') x
#               group by x.id
#              )yy
#   where xx.id = yy.id;

#  insert into public_for_2324.tt_tag_proportion
#   select 'vba' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#     from (
#          select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#           from public_for_2324.posts
#           where posttypeid = '1'
#             and creationdate between '2021-11-30' and '2024-12-02'
#             and tags like '%<vba>%'
#              ) xx,
#          (
#              select x.id, count(x.tag) as tot_cnt
#                 from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                         from public_for_2324.posts
#                       where posttypeid = '1'
#                         and creationdate between '2021-11-30' and '2024-12-02'
#                         and tags like '%<vba>%') x
#               group by x.id
#              )yy
#   where xx.id = yy.id
#   ;

In [2]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
ppp = os.path.abspath('../visualization')
sys.path.insert(1, p)
sys.path.insert(1, pp)
sys.path.insert(1, ppp)

In [3]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf
import string
import re
import config.config as conf
from constants import CONSTANTS
import json

In [4]:
# lang='python'
lang='vba'

In [5]:
conn = psycopg2.connect(dbname  =   conf.database_user['dbname'], 
                        user    =   conf.database_user['user'], 
                        password=   conf.database_user['password'], 
                        host    =   conf.database_user['host'], 
                        port    =   conf.database_user['port'])

In [6]:
lang_tag_dict = {'python' : 'python',
                'cpp': 'c++',
                'java':'java',
                'vba':'vba'
                }

In [7]:
for idx in range(len(CONSTANTS.monthly_timestamps)-1):
    st_dt = CONSTANTS.monthly_timestamps[idx].replace('.','-')
    end_dt = CONSTANTS.monthly_timestamps[idx+1].replace('.','-')

    cur = conn.cursor()
    sql = f"""select  a.id, a.creationdate, a.title, a.tags from public_for_2324.posts a where a.posttypeid = '1' and creationdate >=  '{str(st_dt)}'  and creationdate < '{str(end_dt)}' and tags like '%<{lang_tag_dict[lang]}>%' """
    cur.execute(sql)
    inserted_data = cur.fetchall()
    type(inserted_data)
    cur.close()
    

    dict_q = [{'id' : x[0], 
            'creationdate' : x[1].isoformat(),
            'title' : x[2],
            'tags' : x[3],
            } for x in inserted_data]
    # json_str = json.dumps(dict_q, default=str, ensure_ascii=False, indent=2)

    with open(f"../../visualization/result/tag/run_id_2/data/{lang}/{idx}.json", "w", encoding="utf-8") as f:
        json.dump(dict_q, f, ensure_ascii=False, indent=2)



In [8]:
cur = conn.cursor()
cur.execute(f"""select tag, count(*) as count from public_for_2324.tt_tag_proportion where lang = '{lang_tag_dict[lang]}' group by tag """)
inserted_data = cur.fetchall()
cur.close()
    

dict_q = { x[0]: {"count" : x[1]} for x in inserted_data}
print(type(dict_q))

with open(f"../../visualization/result/tag/run_id_2/data/tag_info_{lang}.json", "w", encoding="utf-8") as f:
    json.dump(dict_q, f, ensure_ascii=False, indent=2)



<class 'dict'>


In [9]:
conn.close()